In [6]:
!pip install python-dotenv

In [2]:
import openai
import pandas as pd
import re
from dotenv import load_dotenv
import os


class ProductRecommendationChatbot:
    def __init__(self, catalog_path='asus_clean.csv'):
        """
        Initialize chatbot with product catalog
        
        Args:
            catalog_path (str): Path to the CSV file containing product catalog
        """
        # Validate catalog path
        if not os.path.exists(catalog_path):
            raise FileNotFoundError(f"Katalog tidak ditemukan di {catalog_path}")
        
        # Read catalog
        self.catalog = pd.read_csv(catalog_path)
        
        # Validate catalog columns
        required_columns = ['NamaBarang', 'Harga', 'Link', 'terjual', 'DeskripsiBarang']
        missing_columns = [col for col in required_columns if col not in self.catalog.columns]
        if missing_columns:
            raise ValueError(f"Kolom berikut hilang dari katalog: {missing_columns}")
        
        # Preprocess catalog for easier searching
        self.catalog['SearchText'] = self.catalog.apply(
            lambda row: f"{row['NamaBarang']} {row['DeskripsiBarang']}".lower(), 
            axis=1
        )

    def parse_query(self, query):
        """
        Parse query to extract specific search parameters
        """
        query_lower = query.lower()
        
        # Price detection
        price_patterns = [
            r'\b(di bawah|kurang dari|<=)\s*([\d.,]+)\s*(ribu|rb|juta)?',
            r'\b(di atas|lebih dari|>=)\s*([\d.,]+)\s*(ribu|rb|juta)?'
        ]
        
        price = None
        price_comparison = None
        for pattern in price_patterns:
            match = re.search(pattern, query_lower)
            if match:
                price_comparison = match.group(1)
                price_str = match.group(2)
                multiplier = match.group(3) or ''
                
                # Convert price
                price = float(price_str.replace(',', ''))
                if 'ribu' in multiplier or 'rb' in multiplier:
                    price *= 1000
                elif 'juta' in multiplier:
                    price *= 1000000
                break
        
        # Specification detection
        spec_keywords = [
            'core i3', 'core i5', 'core i7', 'core i9', 
            'ryzen 3', 'ryzen 5', 'ryzen 7', 'ryzen 9',
            'rtx', 'gtx', 'gaming', 'ssd', 
            r'ram \d+gb'
        ]
        specs = re.findall('|'.join(spec_keywords), query_lower)
        
        return price, price_comparison, specs

    def find_products(self, query):
        """
        Find products matching query parameters
        """
        price, price_comparison, specs = self.parse_query(query)
        
        # Initial filtering
        filtered_catalog = self.catalog.copy()
        
        # Price filtering
        if price is not None:
            if price_comparison in ['di bawah', 'kurang dari', '<=']:
                filtered_catalog = filtered_catalog[filtered_catalog['Harga'] <= price]
            elif price_comparison in ['di atas', 'lebih dari', '>=']:
                filtered_catalog = filtered_catalog[filtered_catalog['Harga'] >= price]
        
        # Specification filtering
        if specs:
            spec_query = ' '.join(specs)
            mask = filtered_catalog['SearchText'].str.contains('|'.join(specs), case=False)
            filtered_catalog = filtered_catalog[mask]
        
        # Sort by relevance (Terjual as a proxy)
        filtered_catalog = filtered_catalog.sort_values('terjual', ascending=False)
        
        return filtered_catalog.head(5)

    def format_product_recommendations(self, products):
        """
        Format product recommendations for response
        """
        if products.empty:
            return "Maaf, tidak ada produk yang sesuai dengan pencarian Anda."
        
        recommendations = []
        for _, product in products.iterrows():
            recommendation = (
                f"• *{product['NamaBarang']}*\n"
                f"  Harga: Rp{int(product['Harga']):,}\n"
                f"  Terjual: {product['terjual']}\n"
                f"  Deskripsi: {product['DeskripsiBarang']}\n"
                f"  Link: {product['Link']}"
            )
            recommendations.append(recommendation)
        
        return "\n\n".join(recommendations)

    def generate_response(self, query):
        """
        Generate AI-powered response using OpenAI
        """
        # Find matching products
        matched_products = self.find_products(query)
        products_text = self.format_product_recommendations(matched_products)
        
        # Prepare system prompt
        system_prompt = """
        Kamu adalah asisten AI untuk rekomendasi produk laptop di toko kami. 
        Gunakan gaya bahasa santai khas penjual Medan, dengan interaksi akrab. Fokus pada membantu pelanggan 
        menemukan laptop yang sesuai kebutuhan.

        Aturan:
        - Selalu berikan rekomendasi berdasarkan produk yang tersedia
        - Gunakan bahasa ramah dan komunikatif
        - Jika tidak ada produk sesuai, sampaikan dengan sopan
        - Bantu pelanggan dengan pertanyaan spesifik tentang laptop
        - Hindari membahas topik di luar katalog produk
        """
        
        # Generate response using GPT
        try:
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": query},
                    {"role": "assistant", "content": f"Produk yang ditemukan:\n{products_text}"},
                    {"role": "user", "content": "Berikan rekomendasi yang sesuai dengan query pelanggan."}
                ],
                max_tokens=300,
                temperature=0.7
            )
            gpt_response = response.choices[0].message.content.strip()
            
            # perubahan
            self.conversation_history.append({"role": "assistant", "content": gpt_response})
            self._save_history()  # Save updated history to file
            # perubahan END
            
            return gpt_response
        
        except Exception as e:
            return f"Maaf, terjadi kesalahan dalam pemrosesan: {str(e)}"

def main():
    # Load environment variables
    load_dotenv()
    openai_api_key = os.getenv('OPENAI_API_KEY')
    # Initialize OpenAI client
    openai.api_key = openai_api_key
    if not openai_api_key:
        print("Silakan atur OPENAI_API_KEY terlebih dahulu di environment variable.")
        return

    try:
        # Inisialisasi chatbot
        chatbot = ProductRecommendationChatbot()
        
        print("Selamat datang! Saya adalah asisten rekomendasi laptop. Ada yang bisa saya bantu?")
        
        while True:
            user_input = input("\nMasukkan pertanyaan Anda (ketik 'keluar' untuk berhenti): ")
            
            if user_input.lower() in ['keluar', 'exit', 'bye']:
                print("Terima kasih! Semoga harimu menyenangkan.")
                break
            
            # Generate and print response
            response = chatbot.generate_response(user_input)
            print("\nRekomendasi:", response)
    
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

if __name__ == "__main__":
    main()

Selamat datang! Saya adalah asisten rekomendasi laptop. Ada yang bisa saya bantu?



Masukkan pertanyaan Anda (ketik 'keluar' untuk berhenti):  laptop kuat untuk ngoding



Rekomendasi: Untuk kegiatan ngoding, saya merekomendasikan laptop ASUS TUF GAMING F15 FX506HC-I535B8T-O11 - Graphite Black. Laptop ini dilengkapi dengan prosesor Intel Core i5-11400H, NVIDIA GeForce RTX 3050, RAM 16GB, dan storage 512GB SSD. Dengan layar 15.6 inci 144Hz, kamu akan mendapatkan performa yang powerful dan visual yang jernih. Selain itu, laptop ini juga memiliki fitur Intelligent Cooling untuk menjaga suhu laptop tetap optimal saat digunakan dalam waktu yang lama. 

Link: [ASUS TUF GAMING F15 FX506HC-I535B8T-O11 - Graphite Black](https://www.tokopedia.com/asus/asus-tuf-gaming-f15-fx506hc-i535b8t-o11-graphite-black?extParam=src%3Dshop%26whid%3D2383689)

Semoga rekomendasi ini sesuai dengan kebutuhan ngoding kamu ya! Jika ada pertanyaan lebih lanjut, jangan ragu untuk bertanya.



Masukkan pertanyaan Anda (ketik 'keluar' untuk berhenti):  harganya berapa tuch?



Rekomendasi: Hai! Berikut adalah rekomendasi laptop berdasarkan harga yang sesuai dengan query pelanggan:

Jika kamu mencari laptop dengan harga sekitar Rp6-7 juta, ASUS VivoBook A516JAO-VIPS356 - Transparent Silver bisa menjadi pilihan yang bagus. Dengan prosesor Intel Core i3, grafis Intel UHD, RAM 4GB, dan storage 512GB, laptop ini cocok untuk kebutuhan sehari-hari seperti browsing, tugas sekolah, dan streaming.

Namun, jika kamu memiliki budget sedikit lebih besar sekitar Rp8,5 juta, ASUS VivoBook Ultra 15 K513EA-OLED351 - Indie Black bisa menjadi pilihan yang menarik. Laptop ini dilengkapi dengan layar OLED FHD, prosesor Intel Core i3, RAM 4GB + ADD ON RAM 8GB, dan storage 512GB. Dengan layar OLED yang brilian, laptop ini cocok untuk multimedia dan tugas produktivitas.

Semoga rekomendasi ini membantu dalam memilih laptop yang sesuai dengan kebutuhan dan budget kamu. Jangan ragu untuk bertanya lebih lanjut jika ada yang perlu kamu klarifikasi ya!



Masukkan pertanyaan Anda (ketik 'keluar' untuk berhenti):  exit


Terima kasih! Semoga harimu menyenangkan.
